In [1]:
cd executible/

/Users/alexeygilman/repos/Cu_transition_time_course-/executible


In [2]:
%run Cu_transition_functionalized.py

In [3]:
import hdbscan
import time
from sklearn import metrics

In [4]:
df1_raw_FM40 = raw_data_cleanup("5G_counts.tsv")


columns = ['5GB1_FM40_T0m_TR2', '5GB1_FM40_T10m_TR3', '5GB1_FM40_T20m_TR2', '5GB1_FM40_T40m_TR1',
           '5GB1_FM40_T60m_TR1', '5GB1_FM40_T90m_TR2', '5GB1_FM40_T150m_TR1_remake', '5GB1_FM40_T180m_TR1']

df2_TPM = TPM_counts(df1_raw_FM40, "start_coord", "end_coord",columns, remove_zero = True)  #TPM counts
df2_TPM_log2 = log_2_transform(df2_TPM, "5GB1_FM40_T0m_TR2","5GB1_FM40_T180m_TR1") #TPM log 2 transformed 
df2_TPM_mean = mean_center(df2_TPM, "5GB1_FM40_T0m_TR2","5GB1_FM40_T180m_TR1") #TPM mean centered 

df3_pearson_r = congruency_table(df2_TPM, "5GB1_FM40_T0m_TR2" , "5GB1_FM40_T180m_TR1", step = df2_TPM.shape[0])
df3_euclidean_mean = euclidean_distance(df2_TPM_mean, "5GB1_FM40_T0m_TR2" , "5GB1_FM40_T180m_TR1")
df3_euclidean_log2 = euclidean_distance(df2_TPM_mean, "5GB1_FM40_T0m_TR2" , "5GB1_FM40_T180m_TR1" )

print("The shape of the TPM table is ", df2_TPM.shape)
print("The shape of the pearson_r matrix is ", df3_pearson_r.shape)


5G_counts.tsv was located in the directory
5G_counts.tsv was imported into dataframe
QC columns were removed from dataframe
All non FM40 data were removed from dataframe
All FM40 columns were sorted by timecourse sequence
Clean-up of raw data complete
The shape of the TPM table is  (4480, 16)
The shape of the pearson_r matrix is  (4480, 4480)


   ### Clustering pearsons_r with HDBSCAN

In [5]:
# Clustering the pearsons_R with N/A vlaues removed 

hdb_t1 = time.time()
hdb_pearson_r = hdbscan.HDBSCAN(metric = "precomputed", min_cluster_size=10).fit(df3_pearson_r)
hdb_pearson_r_labels = hdb_pearson_r.labels_
hdb_elapsed_time = time.time() - hdb_t1
print("time to cluster", hdb_elapsed_time)

time to cluster 4.785267114639282


In [6]:
print(np.unique(hdb_pearson_r_labels)) # unique bins, zero is noise
print(np.bincount(hdb_pearson_r_labels[hdb_pearson_r_labels!=-1])) 

[-1  0  1]
[224  15]


In [24]:
pearson_clusters = {i: np.where(hdb_pearson_r_labels == i)[0] for i in range(2)}

In [23]:
pearson_clusters

{0: array([  75,  109,  124,  125,  208,  217,  236,  237,  250,  278,  281,
         282,  318,  319,  320,  341,  342,  344,  404,  405,  406,  407,
         408,  499,  500,  501,  502,  503,  517,  525,  526,  530,  608,
         657,  660,  661,  663,  665,  666,  669,  673,  683,  686,  687,
         688,  690,  691,  692,  693,  694,  696,  699,  700,  738,  847,
         938,  983,  990,  992,  993,  995, 1028, 1107, 1123, 1151, 1152,
        1153, 1318, 1319, 1328, 1330, 1331, 1342, 1343, 1403, 1425, 1469,
        1503, 1512, 1544, 1550, 1577, 1632, 1633, 1641, 1643, 1804, 1805,
        1820, 1845, 1855, 1866, 1878, 1954, 1960, 1961, 1962, 1989, 2014,
        2110, 2156, 2162, 2165, 2175, 2189, 2218, 2234, 2318, 2332, 2334,
        2336, 2337, 2340, 2380, 2382, 2399, 2458, 2466, 2477, 2478, 2483,
        2506, 2518, 2540, 2586, 2587, 2603, 2605, 2606, 2654, 2725, 2727,
        2743, 2805, 2817, 2818, 2819, 2825, 3062, 3063, 3143, 3297, 3301,
        3363, 3391, 3414, 3437, 348

In [25]:
#pd.set_option('display.height', 500)  #These two commands allow for the display of max of 500 rows - exploring genes
#pd.set_option('display.max_rows', 500) 
df2_TPM.iloc[pearson_clusters[1],:] #the genes that were clustered together [0,1]

,product,type,gene_symbol,locus,start_coord,end_coord,note,translation,5GB1_FM40_T0m_TR2,5GB1_FM40_T10m_TR3,5GB1_FM40_T20m_TR2,5GB1_FM40_T40m_TR1,5GB1_FM40_T60m_TR1,5GB1_FM40_T90m_TR2,5GB1_FM40_T150m_TR1_remake,5GB1_FM40_T180m_TR1
locus_tag,,,,,,,,,,,,,,,,
MBURv2_160283,"putative ferric siderophore transport system, ...",CDS,NaN,MBURv2,3576162,3576686,Evidence 3 : Function proposed based on presen...,NaN,118.243568,112.275736,110.964306,100.836391,77.197657,70.032566,73.893920,95.436505
MBURv2_20180,"ferrous iron transporter, protein A",CDS,feoA,MBURv2,281921,282154,Evidence 2a : Function of homologous gene expe...,NaN,210.424105,158.632845,181.080545,126.342956,61.051857,70.194605,95.464947,118.980515
MBURv2_20181,"fused ferrous iron transporter, protein B: GTP...",CDS,feoB,MBURv2,282154,284478,Evidence 2a : Function of homologous gene expe...,NaN,95.819838,86.744847,91.146440,71.621996,32.461622,39.674872,55.479556,64.157192
MBURv2_20183,conserved protein of unknown function,CDS,NaN,MBURv2,284953,286617,Evidence 4 : Homologs of previously reported g...,NaN,92.724781,77.558180,86.194789,70.352941,49.814896,47.463966,55.260844,59.371024
MBURv2_210346,Histidine kinase,CDS,NaN,MBURv2,4474339,4475010,NaN,NaN,70.518396,68.822867,66.867096,63.493064,49.910520,54.874815,53.845876,59.197819
MBURv2_240071,conserved protein of unknown function,CDS,NaN,MBURv2,4822700,4823671,Evidence 4 : Homologs of previously reported g...,NaN,115.521215,125.266171,122.873477,171.859978,242.070964,230.034969,215.806091,173.416287
MBURv2_240072,conserved protein of unknown function,CDS,NaN,MBURv2,4823671,4825122,Evidence 4 : Homologs of previously reported g...,NaN,152.884221,166.004637,159.391668,244.378006,339.549453,333.040188,298.420330,260.795367
MBURv2_30040,conserved exported protein of unknown function,CDS,NaN,MBURv2,648480,649751,Evidence 4 : Homologs of previously reported g...,NaN,346.196665,456.748233,416.782502,555.641953,1007.956458,965.279974,855.040811,766.323303
MBURv2_30115,Integrase/recombinase,CDS,int,MBURv2,749393,750415,NaN,NaN,18.954208,19.738377,18.230891,15.962341,11.687684,12.175099,14.009989,14.678014


Looks like there are two clusters, some expression and zero expression across samples.

   ### Clustering mean centered euclidean distance with with HDBSCAN

In [27]:
df3_euclidean_mean.hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x113686b00>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x115baf9e8>, ...,
        <matplotlib.axes._subplots.AxesSubplot object at 0x118532be0>, ...,
        <matplotlib.axes._subplots.AxesSubplot object at 0x11a150518>, ...,
       ..., 
        <matplotlib.axes._subplots.AxesSubplot object at 0x1aadc8a90>, ...,
        <matplotlib.axes._subplots.AxesSubplot object at 0x1ad458b38>, ...,
        <matplotlib.axes._subplots.AxesSubplot object at 0x1aefcf0b8>, ...,
        <matplotlib.axes._subplots.AxesSubplot object at 0x1b0c58da0>]], dtype=object)

In [10]:
# Clustering the mean centered euclidean distance of TPM counts 

hdb_t1 = time.time()
hdb_euclidean_mean = hdbscan.HDBSCAN(metric = "precomputed", min_cluster_size=10).fit(df3_euclidean_mean)
hdb_euclidean_mean_labels = hdb_euclidean_mean.labels_
hdb_elapsed_time = time.time() - hdb_t1
print("time to cluster", hdb_elapsed_time)

time to cluster 6.022979021072388


In [11]:
print(np.unique(hdb_euclidean_mean_labels))
print(np.bincount(hdb_euclidean_mean_labels[hdb_euclidean_mean_labels!=-1]))

[-1]
[]


In [12]:
euclidean_mean_clusters = {i: np.where(hdb_euclidean_mean_labels == i)[0] for i in range(2)}
df2_TPM.iloc[euclidean_mean_clusters[1],:]

,product,type,gene_symbol,locus,start_coord,end_coord,note,translation,5GB1_FM40_T0m_TR2,5GB1_FM40_T10m_TR3,5GB1_FM40_T20m_TR2,5GB1_FM40_T40m_TR1,5GB1_FM40_T60m_TR1,5GB1_FM40_T90m_TR2,5GB1_FM40_T150m_TR1_remake,5GB1_FM40_T180m_TR1
locus_tag,,,,,,,,,,,,,,,,


Looks like 2 clusters - both with zero expression. 

looks like wether it is a numpy array or pandas dataframe, the result is the same. lets now try to get index of the clustered points. 

   ### Clustering log transformed euclidean distance with with HDBSCAN

In [28]:
df3_euclidean_log2

locus_tag,MBURv2_100001,MBURv2_100002,MBURv2_100003,MBURv2_10001,MBURv2_10002,MBURv2_10003,MBURv2_10004,MBURv2_10005,MBURv2_10006,MBURv2_10007,...,MBURv2_tRNA4,MBURv2_tRNA40,MBURv2_tRNA41,MBURv2_tRNA42,MBURv2_tRNA43,MBURv2_tRNA44,MBURv2_tRNA5,MBURv2_tRNA6,MBURv2_tRNA7,MBURv2_tRNA8
locus_tag,,,,,,,,,,,,,,,,,,,,,
MBURv2_100001,0.000000,5.223666,7.344067,14.843396,29.298972,21.745323,5.826292,5.193588,21.612292,13.313414,...,26.117059,419.208451,302.921397,208.678333,61.568151,34.697223,345.013663,373.585068,158.429465,560.223957
MBURv2_100002,5.223666,0.000000,7.404187,11.260013,28.311112,25.650899,4.964426,5.368636,21.517591,12.377645,...,29.484939,418.969169,302.496445,210.394089,62.091403,35.519929,344.939793,371.929211,160.390886,558.506612
MBURv2_100003,7.344067,7.404187,0.000000,11.144207,33.202231,23.468108,6.460714,8.801506,26.465912,17.807287,...,25.927345,422.591759,306.608212,211.532877,63.378991,38.293495,348.362274,376.123186,162.422325,562.705638
MBURv2_10001,14.843396,11.260013,11.144207,0.000000,36.561595,32.567248,11.834242,15.719222,30.130630,21.260764,...,31.907818,425.770363,308.543398,217.381023,69.715512,44.564529,351.601790,377.287805,169.289609,563.031356
MBURv2_10002,29.298972,28.311112,33.202231,36.561595,0.000000,42.318096,29.711239,26.956130,12.577875,17.564065,...,52.600629,392.874864,279.141782,191.489245,46.816720,22.642093,322.403709,350.375007,146.718629,533.465735
MBURv2_10003,21.745323,25.650899,23.468108,32.567248,42.318096,0.000000,24.672872,23.686298,35.243224,30.588730,...,20.153934,426.626657,309.720338,209.958893,67.288269,42.042220,349.383536,381.680300,153.974846,569.166693
MBURv2_10004,5.826292,4.964426,6.460714,11.834242,29.711239,24.672872,0.000000,6.982259,22.569002,14.056489,...,26.930440,419.792728,302.463486,209.973733,63.109560,34.868167,344.580788,372.060187,159.402700,558.712223
MBURv2_10005,5.193588,5.368636,8.801506,15.719222,26.956130,23.686298,6.982259,0.000000,21.278581,12.748837,...,29.101352,416.368657,300.264236,207.267639,57.952570,31.571145,341.959065,369.408899,155.940651,557.035614
MBURv2_10006,21.612292,21.517591,26.465912,30.130630,12.577875,35.243224,22.569002,21.278581,0.000000,9.313694,...,43.521775,402.952193,288.522213,198.545200,55.964345,28.080435,332.237623,360.649670,152.974739,543.894001


In [13]:
# Clustering the log2 transformed euclidean distance of TPM counts 

hdb_t1 = time.time()
hdb_euclidean_log2 = hdbscan.HDBSCAN(metric = "precomputed", min_cluster_size=10).fit(df3_euclidean_log2)
hdb_euclidean_log2_labels = hdb_euclidean_log2.labels_
hdb_elapsed_time = time.time() - hdb_t1
print("time to cluster", hdb_elapsed_time)

time to cluster 4.981569051742554


In [14]:
print(np.unique(hdb_euclidean_log2_labels))
print(np.bincount(hdb_euclidean_log2_labels[hdb_euclidean_log2_labels!=-1]))

[-1]
[]


In [15]:
euclidean_log2_clusters = {i: np.where(hdb_euclidean_log2_labels == i)[0] for i in range(2)}
df2_TPM.iloc[euclidean_log2_clusters[1],:]

,product,type,gene_symbol,locus,start_coord,end_coord,note,translation,5GB1_FM40_T0m_TR2,5GB1_FM40_T10m_TR3,5GB1_FM40_T20m_TR2,5GB1_FM40_T40m_TR1,5GB1_FM40_T60m_TR1,5GB1_FM40_T90m_TR2,5GB1_FM40_T150m_TR1_remake,5GB1_FM40_T180m_TR1
locus_tag,,,,,,,,,,,,,,,,


### Clustering using built-in HDBSCAN euclidean distance metric (mean centered and scaled to unit variance) 

In [16]:
df2_TPM_values = df2_TPM.loc[:,"5GB1_FM40_T0m_TR2":"5GB1_FM40_T180m_TR1"] #isolating the data values 
    
df2_TPM_values_T = df2_TPM_values.T #transposing the data
standard_scaler = StandardScaler()

TPM_counts_mean_centered = standard_scaler.fit_transform(df2_TPM_values_T) #mean centering the data 
TPM_counts_mean_centered = pd.DataFrame(TPM_counts_mean_centered) #back to Dataframe

#transposing back to original form and reincerting indeces and columns 
my_index = df2_TPM_values.index
my_columns = df2_TPM_values.columns

TPM_counts_mean_centered = TPM_counts_mean_centered.T
TPM_counts_mean_centered.set_index(my_index, inplace=True)
TPM_counts_mean_centered.columns = my_columns

In [17]:
# Clustering the pearsons_R with N/A vlaues removed 

hdb_t1 = time.time()
hdb_euclidean = hdbscan.HDBSCAN(metric = "euclidean", min_cluster_size=5).fit(TPM_counts_mean_centered)
hdb_euclidean_labels = hdb_euclidean.labels_
hdb_elapsed_time = time.time() - hdb_t1
print("time to cluster", hdb_elapsed_time)

time to cluster 1.2530548572540283


In [18]:
print(np.unique(hdb_euclidean_labels))
print(np.bincount(hdb_euclidean_labels[hdb_euclidean_labels!=-1]))

[-1  0  1  2  3  4  5  6]
[   5    6    9  569    6 1538    6]


###### lets look at some clusters
Euclidean_standard_scaled_clusters = {i: np.where(hdb_euclidean_labels == i)[0] for i in range(7)}
df2_TPM.iloc[Euclidean_standard_scaled_clusters[0],:]

In [30]:
Euclidean_standard_scaled_clusters = {i: np.where(hdb_euclidean_labels == i)[0] for i in range(7)}
df2_TPM.iloc[Euclidean_standard_scaled_clusters[1],:]

,product,type,gene_symbol,locus,start_coord,end_coord,note,translation,5GB1_FM40_T0m_TR2,5GB1_FM40_T10m_TR3,5GB1_FM40_T20m_TR2,5GB1_FM40_T40m_TR1,5GB1_FM40_T60m_TR1,5GB1_FM40_T90m_TR2,5GB1_FM40_T150m_TR1_remake,5GB1_FM40_T180m_TR1
locus_tag,,,,,,,,,,,,,,,,
MBURv2_130823,conserved hypothetical protein; putative membr...,CDS,NaN,MBURv2,3057071,3057622,Evidence 4 : Homologs of previously reported g...,NaN,54.074431,42.385029,49.380410,45.924264,54.430108,52.911230,42.512312,49.776228
MBURv2_160196,DNA replication and repair protein RecF,CDS,recF,MBURv2,3476509,3477588,NaN,NaN,39.716085,33.586759,38.475054,34.019740,41.174621,39.432168,34.425668,39.151422
MBURv2_210033,histidyl tRNA synthetase,CDS,hisS,MBURv2,4129786,4131057,Evidence 2a : Function of homologous gene expe...,NaN,184.219401,153.374251,180.859518,160.574002,190.176907,182.024224,158.874769,178.341500
MBURv2_260039,hydroxymethylbilane synthase,CDS,hemC,MBURv2,5063172,5064098,Evidence 2a : Function of homologous gene expe...,NaN,186.700990,151.238240,172.944983,160.882580,188.444713,184.348282,153.582663,184.702961
MBURv2_60016,"carbamoyl phosphate synthetase small subunit, ...",CDS,carA,MBURv2,1374314,1375447,Evidence 2a : Function of homologous gene expe...,NaN,417.342736,361.297562,405.602208,364.758505,454.132299,433.434899,357.685236,404.088838
MBURv2_60050,Na+/H+ antiporter NhaD,CDS,nhaD,MBURv2,1419064,1420479,NaN,NaN,196.773053,167.449917,183.707146,168.936189,219.249940,198.774079,178.440470,188.004395


##### Euclidean_standard_scaled_clusters

### Clustering log2 transformed data using built-in HDBSCAN euclidean distance metric (mean centered and scaled to unit variance) 

In [19]:
df2_TPM_log2_scale= df2_TPM_log2.T #transposing the data
standard_scaler = StandardScaler()

TPM_log2_mean_scaled = standard_scaler.fit_transform(df2_TPM_log2_scale) #mean centering the data 
TPM_log2_mean_scaled = pd.DataFrame(TPM_log2_mean_scaled) #back to Dataframe

#transposing back to original form and reincerting indeces and columns 
my_index = df2_TPM_values.index
my_columns = df2_TPM_values.columns

TPM_log2_mean_scaled = TPM_log2_mean_scaled.T
TPM_log2_mean_scaled.set_index(my_index, inplace=True)
TPM_log2_mean_scaled.columns = my_columns

In [20]:
# Clustering the pearsons_R with N/A vlaues removed 

hdb_t1 = time.time()
hdb_log2_euclidean = hdbscan.HDBSCAN(metric = "euclidean", min_cluster_size=5).fit(TPM_log2_mean_scaled)
hdb_log2_euclidean = hdb_log2_euclidean.labels_
hdb_elapsed_time = time.time() - hdb_t1
print("time to cluster", hdb_elapsed_time)

time to cluster 1.0190801620483398


In [21]:
print(np.unique(hdb_log2_euclidean))
print(np.bincount(hdb_log2_euclidean[hdb_log2_euclidean!=-1]))

[0 1]
[   8 4472]
